Install dependencies

In [ ]:
!pip install pandas
!pip install numpy
!pip install jupyter-dash
!pip install matplotlib
!pip install psutil
!pip install seaborn

Import requirements

In [ ]:
import pandas as pd
import os.path

import seaborn as sns
sns.set_palette("muted")
sns.set(style="ticks", color_codes=True)

# Import chart data

Load global charts and track features

In [ ]:
featureData = pd.read_csv('data/track_features.csv')

Function to import data by region group

In [ ]:
def load_charts_for_codes(codes):
    temp_df = pd.DataFrame()
    for code in codes["code"]:
        if os.path.isfile("data/chart_{}.csv".format(code.lower())):
            temp_df = temp_df.append(pd.read_csv("data/chart_{}.csv".format(code.lower())))
    return temp_df

Load regional chart data

In [ ]:
#Europe
codes = pd.read_csv("data/EuropeCode.csv")
chartDataEurope = load_charts_for_codes(codes)

#Asia
codes = pd.read_csv('data/AsiaCode.csv')
chartDataAsia = load_charts_for_codes(codes)

#NorthAmerica
codes = pd.read_csv('data/NorthAmericaCode.csv')
chartDataNorthAmerica = load_charts_for_codes(codes)

#SouthAmerica
codes = pd.read_csv('data/SouthAmericaCode.csv')
chartDataSouthAmerica = load_charts_for_codes(codes)

#Africa
codes = pd.read_csv('data/AfricaCode.csv')
chartDataAfrica = load_charts_for_codes(codes)

#Oceania
codes = pd.read_csv('data/OcianiaCode.csv')
chartDataOceania = load_charts_for_codes(codes)

# Merging

## prepare data of countries

Function to join chart tracks and track features

In [ ]:
def join_chart_features_continents(chart):
    info = chart.merge(featureData, left_on='track_id', right_on='track_id')
    info = info.groupby('date', as_index=False).mean()
    return info

Join features and tracks per region and continent

In [ ]:
meanEurope = join_chart_features_continents(chartDataEurope)
meanAsia = join_chart_features_continents(chartDataAsia)
meanNorthAmerica = join_chart_features_continents(chartDataNorthAmerica)
meanSouthAmerica = join_chart_features_continents(chartDataSouthAmerica)
meanOceania = join_chart_features_continents(chartDataOceania)

Function to join region data

In [ ]:
def join_region_data(data, region):
    tempDf = data
    tempDf["region"] = region
    tempDf = tempDf.drop("Position", axis=1)
    tempDf = tempDf.drop("date", axis=1)
    return tempDf

In [ ]:
frames = [
    join_region_data(meanEurope, "Europe"),
    join_region_data(meanAsia, "Asia"),
    join_region_data(meanNorthAmerica, "North America"),
    join_region_data(meanSouthAmerica, "South America"),
    join_region_data(meanOceania, "Oceania")
]

regionDf = pd.concat(frames)

In [ ]:
temp = pd.DataFrame(list(
    zip(
        regionDf["region"],
        regionDf["Streams"],
        regionDf["danceability"],
        regionDf["energy"],
        regionDf["key"],
        regionDf["loudness"],
        regionDf["mode"],
        regionDf["speechiness"],
        regionDf["acousticness"],
        regionDf["instrumentalness"],
        regionDf["liveness"],
        regionDf["valence"],
        regionDf["tempo"],
        regionDf["duration_ms"],
        regionDf["time_signature"]
    )),
    columns=[
        'Region',
        "Streams",
        'Danceability',
        'Energy',
        'Key',
        "Loudness",
        'Mode',
        'Speechiness',
        'Acousticness',
        'Instrumentalness',
        'Liveness',
        'Valence',
        'Tempo',
        'Duration',
        'Time'
    ]
)

In [ ]:
plot = sns.pairplot(temp, hue="Region", palette="muted")
plot.savefig("output.png")